In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import astropy as apy
import sklearn.neighbors as skn
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord

In [2]:
csv = pd.read_csv('../Experiment/Experiment_F3W2_95_5/FIRST_Cata_Images.csv')

/Users/gal16b/miniconda3/envs/pink_e3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
csv.columns.tolist()
ra = np.deg2rad(csv['radio.ra'])
dec = np.deg2rad(csv['radio.dec'])
points = np.array((ra, dec)).T
index = csv.index

In [4]:
kdtree = skn.BallTree(points, metric='haversine')

In [5]:
dist, ind = kdtree.query(points[2].reshape((1,-1)), k=3)

In [6]:
dist[0,1]*u.radian.to(u.arcsecond)

57.50313205867317

In [7]:
ra[ind[0,1]]*u.radian.to(u.deg), dec[ind[0,1]]*u.radian.to(u.deg)

(164.226080844, 23.3930434713)

In [8]:
csv.iloc[ind[0,1]][['radio.ra', 'radio.dec']]

radio.ra     164.226
radio.dec     23.393
Name: 3, dtype: object

In [9]:
csv.iloc[2][['radio.ra', 'radio.dec']]

radio.ra     164.214
radio.dec    23.3822
Name: 2, dtype: object

In [10]:
ra[2]*u.radian.to(u.deg), dec[2]*u.radian.to(u.deg)

(164.21399903200003, 23.382185861700002)

In [11]:
a = SkyCoord(ra=164.226080844*u.deg, dec=23.3930434713*u.deg, frame='icrs')
b = SkyCoord(ra=164.21399903200003*u.deg, dec=23.382185861700002*u.deg, frame='icrs')

a.separation(b)

<Angle 0.01551958 deg>

In [12]:
ccc = b.spherical_offsets_to(a)*u.radian

In [13]:
ccc

<Quantity [0.00019353, 0.00018951] rad>

In [14]:
(2*np.arcsin(np.sqrt(np.sin(ccc[0] * 0.5)**2 + 
             np.cos(a.ra.to(u.radian)) * np.cos(b.ra.to(u.radian))*np.sin(ccc[1]*0.5)**2))).to(u.arcsecond)

<Quantity 54.84993955 arcsec>

In [17]:
from astropy.coordinates.angle_utilities import angular_separation

angular_separation(a.ra.to(u.radian), a.dec.to(u.radian),b.ra.to(u.radian), b.dec.to(u.radian)).to(u.arcsecond)

<Quantity 55.8705045 arcsec>

In [52]:
def sep(a, b):
    """ Code modified from astropy.coordinates.angle_utils.angular_separation
    """
    # Long is 0, lat is 1
    long = 0
    lat = 1
    print(a)
    sdlon = np.sin(b[long,:] - a[long,:])
    cdlon = np.cos(b[long,:] - a[long,:])
    slat1 = np.sin(a[lat,:])
    slat2 = np.sin(b[lat,:])
    clat1 = np.cos(a[lat,:])
    clat2 = np.cos(b[lat,:])
    
    num1 = clat2 * sdlon
    num2 = clat1 * slat2 - slat1 * clat2 * cdlon
    denominator = slat1 * slat2 + clat1 * clat2 * cdlon

    return np.arctan2(np.hypot(num1, num2), denominator)

def sep2(a, b):
    """ Code modified from astropy.coordinates.angle_utils.angular_separation
    """
    sdlon = np.sin(b[0] - a[0])
    cdlon = np.cos(b[0] - a[0])
    slat1 = np.sin(a[1])
    slat2 = np.sin(b[1])
    clat1 = np.cos(a[1])
    clat2 = np.cos(b[1])
    
    num1 = clat2 * sdlon
    num2 = clat1 * slat2 - slat1 * clat2 * cdlon
    denominator = slat1 * slat2 + clat1 * clat2 * cdlon

    return np.arctan2(np.hypot(num1, num2), denominator)

In [53]:
sep(np.array([[a.ra.to(u.radian).value], [a.dec.to(u.radian).value]]),
    np.array([[b.ra.to(u.radian).value], [b.dec.to(u.radian).value]]))[0]*u.radian.to(u.arcsecond)

[[2.86628583]
 [0.40828563]]


55.870504502216846

In [54]:
from sklearn.neighbors import DistanceMetric as dm

In [55]:
kdtree = skn.BallTree(points, metric='pyfunc', func=sep2)

In [56]:
dist, ind = kdtree.query(points[2].reshape((1,-1)), k=3)
dist[0,1]*u.radian.to(u.arcsecond)


55.87050450228054